## Creating a User-level dataset for analysing user insights

This is a notebook started by Richie with the goal to pull data on individual subscribers. The goal is one row per subscriber, with info about them, like their gender, age, but also calculated values like average call duration and number of calls. This data can be used for all sorts of user analysis, including cluster analysis.

In [1]:
import os
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#Set environment variable. 
#Note: CHANGE THIS TO WHERE ON YOUR COMPUTER THE JSON FILE IS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='G:\My Drive\data science\DataDives\\viamo_api_key.json'

Bigquery_client = bigquery.Client()

## Here is how I created the dataset

In [3]:
user_data_base = pd.read_gbq('''select subscriber_id, 
            min(age) as age, 
            min(gender) as gender, 
            count(distinct(call_id)) as n_calls,
            count(distinct(block_theme)) as n_themes,
            count(distinct(block_topic)) as n_topics
    FROM `viamo-datakind.datadive.321_sessions_1122` 
    WHERE organization_country = 'Uganda' 
    GROUP BY subscriber_id''')
print(user_data_base.shape)
print(user_data_base.describe().round(3))

KeyboardInterrupt: 

In [ ]:
median_calls_per_user = pd.read_gbq('''SELECT distinct(subscriber_id), 
PERCENTILE_CONT(duration_listened_seconds, 0.5) OVER(PARTITION BY subscriber_id) as median_call_duration
FROM `viamo-datakind.datadive.321_sessions_1122`
WHERE organization_country = 'Uganda' 
''')
median_calls_per_user.head()

In [ ]:
user_data2 = user_data_base.merge(median_calls_per_user, on = 'subscriber_id', how = 'left')

In [ ]:
print(user_data2['gender'].unique())
print(user_data2['age'].unique())
print(user_data2['median_call_duration'].isna().mean())

In [ ]:
user_data2['gender_numeric'] = user_data2['gender']\
    .replace(['male','female'], [1,2], inplace=False)\
        .fillna(-1)

user_data2['age_numeric'] = user_data2['age']\
    .replace(['under_18', '18_24','25_34','35_44','over_44'], [1, 2, 3, 4, 5], inplace=False)\
        .fillna(-1)

user_data2['median_call_duration']\
    .fillna(-1, inplace = True)

In [ ]:
fav_theme = pd.read_gbq('''with ordered_themes_by_users as (select subscriber_id, block_theme, ROW_NUMBER() OVER (PARTITION BY subscriber_id ORDER BY COUNT(block_theme) DESC) rn
FROM `viamo-datakind.datadive.321_sessions_1122`
WHERE organization_country = 'Uganda'
group by subscriber_id, block_theme)
select subscriber_id, block_theme as fav_theme from ordered_themes_by_users where rn = 1
''')

second_fav_theme = pd.read_gbq('''with ordered_themes_by_users as (select subscriber_id, block_theme, ROW_NUMBER() OVER (PARTITION BY subscriber_id ORDER BY COUNT(block_theme) DESC) rn
FROM `viamo-datakind.datadive.321_sessions_1122`
WHERE organization_country = 'Uganda'
group by subscriber_id, block_theme)
select subscriber_id, block_theme as second_fav_theme from ordered_themes_by_users where rn = 2
''')

In [ ]:
fav_topic = pd.read_gbq('''with ordered_topics_by_users as (select subscriber_id, block_topic, ROW_NUMBER() OVER (PARTITION BY subscriber_id ORDER BY COUNT(block_topic) DESC) rn
FROM `viamo-datakind.datadive.321_sessions_1122`
WHERE organization_country = 'Uganda'
group by subscriber_id, block_topic)
select subscriber_id, block_topic as fav_topic from ordered_topics_by_users where rn = 1
''')

second_fav_topic = pd.read_gbq('''with ordered_topics_by_users as (select subscriber_id, block_topic, ROW_NUMBER() OVER (PARTITION BY subscriber_id ORDER BY COUNT(block_topic) DESC) rn
FROM `viamo-datakind.datadive.321_sessions_1122`
WHERE organization_country = 'Uganda'
group by subscriber_id, block_topic)
select subscriber_id, block_topic as second_fav_topic from ordered_topics_by_users where rn = 2
''')

In [ ]:
user_data3 = user_data2\
.merge(fav_theme,           on = 'subscriber_id', how = 'left')\
.merge(second_fav_theme,    on = 'subscriber_id', how = 'left')\
.merge(fav_topic,           on = 'subscriber_id', how = 'left')\
.merge(second_fav_topic,    on = 'subscriber_id', how = 'left')

In [ ]:
print(user_data3['fav_theme'].unique())
print('\n')
print(user_data3['second_fav_theme'].unique())
print('\n')
print(user_data3['fav_topic'].unique())
print('\n')
print(user_data3['second_fav_topic'].unique())

In [ ]:
# For Mali
user_data3['fav_theme_numeric'] = user_data3['fav_theme']\
    .replace(['', 'news','health','gender','financial','ed','health,nutrition','security','rights','ag'], [-1, 1, 2, 3, 4,5,6,7,8,9], inplace=False)\
        .fillna(-1)
user_data3['second_fav_theme_numeric'] = user_data3['second_fav_theme']\
    .replace(['', 'news','health','gender','financial','ed','health,nutrition','security','rights','ag','games','financial,rights','nutrition'], [-1, 1, 2, 3, 4,5,6,7,8,9,10,11,12], inplace=False)\
        .fillna(-1)

user_data3['fav_topic_numeric'] = user_data3['fav_topic']\
    .replace(['', 'livestock','environment','coronavirus,malaria','coronavirus','malaria,ncd','coronavirus,malaria,ncd','malaria'], [-1, 1, 2, 3, 4,5,6,7], inplace=False)\
        .fillna(-1)
user_data3['second_fav_topic_numeric'] = user_data3['second_fav_topic']\
    .replace(['', 'livestock','environment','coronavirus,malaria','coronavirus','malaria,ncd','coronavirus,malaria,ncd','malaria','gbv','maternal'], [-1, 1, 2, 3, 4,5,6,7,8,9], inplace=False)\
        .fillna(-1)

In [ ]:
# For Uganda
user_data3['fav_theme_numeric'] = user_data3['fav_theme']\
    .replace(['', 'weather','health','ag','vas','news','games','financial','drama'], [-1, 1, 2, 3, 4,5,6,7,8], inplace=False)\
        .fillna(-1)
user_data3['second_fav_theme_numeric'] = user_data3['second_fav_theme']\
    .replace(['', 'weather','health','ag','vas','news','games','financial','drama', 'gender','financial,games','ag,games'], [-1, 1, 2, 3, 4,5,6,7,8,9,10,11], inplace=False)\
        .fillna(-1)

user_data3['fav_topic_numeric'] = user_data3['fav_topic']\
    .replace(['', 'coronavirus','srh','beans','rice','maize','hiv,srh','cassava'], [-1, 1, 2, 3, 4,5,6,7], inplace=False)\
        .fillna(-1)
user_data3['second_fav_topic_numeric'] = user_data3['second_fav_topic']\
    .replace(['', 'coronavirus','srh','beans','rice','maize','hiv,srh','cassava','livestock','hiv'], [-1, 1, 2, 3, 4,5,6,7,8,9], inplace=False)\
        .fillna(-1)

In [ ]:
# For Nigeria TO DO
user_data3['fav_theme_numeric'] = user_data3['fav_theme']\
    .replace(['', 'weather','health','ag','vas','news','games','financial','drama'], [-1, 1, 2, 3, 4,5,6,7,8], inplace=False)\
        .fillna(-1)
user_data3['second_fav_theme_numeric'] = user_data3['second_fav_theme']\
    .replace(['', 'weather','health','ag','vas','news','games','financial','drama', 'gender','financial,games','ag,games'], [-1, 1, 2, 3, 4,5,6,7,8,9,10,11], inplace=False)\
        .fillna(-1)

user_data3['fav_topic_numeric'] = user_data3['fav_topic']\
    .replace(['', 'coronavirus','srh','beans','rice','maize','hiv,srh','cassava'], [-1, 1, 2, 3, 4,5,6,7], inplace=False)\
        .fillna(-1)
user_data3['second_fav_topic_numeric'] = user_data3['second_fav_topic']\
    .replace(['', 'coronavirus','srh','beans','rice','maize','hiv,srh','cassava','livestock','hiv'], [-1, 1, 2, 3, 4,5,6,7,8,9], inplace=False)\
        .fillna(-1)

In [ ]:
user_data3.to_parquet('Viamo_user_data_Uganda.parquet')